**Imports**

In [28]:
import numpy as np
import pandas as pd

**Reading data**

In [61]:
def parse_data(file_path): # Function for parsing .txt with non-numerical string
    data_sets = {}  # Dictionary to hold the data sets
    current_set = None

    errors = 0

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith("Data set"):  # New data set
                current_set = line  # Update the current data set
                data_sets[current_set] = []  # Initialize an empty list for this set
            elif line:  # Non-empty line, add data to current data set
                try:
                    x, y = map(float, line.split('\t'))
                    data_sets[current_set].append([x, y])
                except Exception as e: # An error usually appears due to the initial string line
                    errors += 1

    return data_sets , errors # Get the data sets as a dictionary

data_sets , errors = parse_data('FranksNumbers.txt') # Get the data sets

**Compute means and variances**

In [93]:
means = []
variances = []

for data in data_sets.values():
    
    x_values = [item[0] for item in data]
    y_values = [item[1] for item in data]
    
    means.append( np.average(y_values,weights=x_values)) 
    variances.append( np.var(y_values,ddof=0))
    
print(means)
print(variances)

[8.056565656565656, 8.056464646464647, 8.055252525252524, 8.056363636363637, 8.056161616161615]
[3.7520628099173554, 3.752390082644628, 3.747836363636364, 3.7484082644628103, 3.7501745351239673]


**Chi2**

In [72]:
def test_function(x):
    return x*0.48 + 3.02

In [107]:
chi_values = []

for data in data_sets.values():
    
    x_values = [item[0] for item in data]
    y_values = [item[1] for item in data]
    
    obs_y_values = [test_function(x) for x in x_values] # Get the y-values from function
    
    sy_values_1 = np.sqrt(obs_y_values) # Get uncertainties from Poisson error
    sy_values_2 = [1.22 for x in range(len(y_values))] # Get uncertainties from 1.22

    chi2_1 = 0 # Chi2 using Poisson error
    chi2_2 = 0 # Chi2 using 1.22
        
    for y_2 , y_1 , sy_1 in zip(y_values,obs_y_values,sy_values_1):
        chi2_1 += ((y_2 - y_1)**2) / ( sy_1**2 ) # Compute Chi (Poisson)
        
    for y_2 , y_1 , sy_1 in zip(y_values,obs_y_values,sy_values_2):
        chi2_2 += ((y_2 - y_1)**2) / ( sy_1**2 ) # Compute Chi (1.22)
    
    red_chi2_1 = chi2_1 / (len(x_values)-2)
    red_chi2_2 = chi2_2 / (len(x_values)-2)
    
    chi_values.append([chi2_1,chi2_2,red_chi2_1,red_chi2_2])

[1.5545390861808746, 9.460225745767271, 0.1727265651312083, 1.0511361939741413]

**Summed up**

In [119]:
count = 1

for mean,variance,chi2 in zip(means,variances,chi_values):
    
    print(f'For Data Set {count};')
    print(f'mean is {mean}')
    print(f'Variance is {variance}')
    print(f'Chi2 (sigma = sqrt(y)) is {chi2[0]}')
    print(f'Chi2 (sigma = 1.22) is {chi2[1]}')
    # print(f'Redcued chi2 (sigma = sqrt(y)) is {chi2[2]}')
    # print(f'Reduced chi2 (sigma = 1.22) is {chi2[3]}')
    
    count += 1

For Data Set 1;
mean is 8.056565656565656
Variance is 3.7520628099173554
Chi2 (sigma = sqrt(y)) is 1.8868253038459033
Chi2 (sigma = 1.22) is 9.467817790916422
For Data Set 2;
mean is 8.056464646464647
Variance is 3.752390082644628
Chi2 (sigma = sqrt(y)) is 2.0715758463593357
Chi2 (sigma = 1.22) is 9.47668637463048
For Data Set 3;
mean is 8.055252525252524
Variance is 3.747836363636364
Chi2 (sigma = sqrt(y)) is 1.5545390861808746
Chi2 (sigma = 1.22) is 9.460225745767271
For Data Set 4;
mean is 8.056363636363637
Variance is 3.7484082644628103
Chi2 (sigma = sqrt(y)) is 2.0429349282664355
Chi2 (sigma = 1.22) is 9.453708680462244
For Data Set 5;
mean is 8.056161616161615
Variance is 3.7501745351239673
Chi2 (sigma = sqrt(y)) is 7.55587516465255
Chi2 (sigma = 1.22) is 37.85843859177642
